In [68]:
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [2]:
# prepare probe annotation file
! ls ./data/AHBA_expression/normalized_microarray_donor0*/probes.csv |xargs cat |sort |uniq |grep -v "^probe" >./data/probes.csv

# 0.Preprocessing

In [166]:
# load probe information
file_probe_annot = "./data/probes.csv"
probe_annot = pd.read_csv("./data/probes.csv", sep=",", header=None)
probe_annot.columns = ["probe_id", "probe_name", "gene_id", "gene_symbol", "gene_name", "entrez_id", "chromosome"]
probe_annot.shape

(58692, 7)

In [167]:
# load re-annotation information
file_probe_reannot = "./data/AHBA_expression/reannotated.csv.gz"
probe_reannot = pd.read_csv(file_probe_reannot)
np.bincount(probe_reannot['compare']), probe_reannot.shape
# mismatch(0), match(1), introduced(2), not Given(3)

(array([ 1287, 42383,  2151,    71]), (45892, 5))

In [168]:
# load sample information
data = {}
for subj in range(1, 7):
    file_microarray = f"./data/AHBA_expression/normalized_microarray_donor0{subj}/MicroarrayExpression.csv"
    file_noise = f"./data/AHBA_expression/normalized_microarray_donor0{subj}/PACall.csv"
    file_annot = f"./data/AHBA_expression/normalized_microarray_donor0{subj}/SampleAnnot.csv"

    data[subj] = {"exprs" : pd.read_csv(file_microarray, header=None, index_col=0),
                  "noise" : pd.read_csv(file_noise, header=None, index_col=0),
                  "annot" : pd.read_csv(file_annot)}
    data[subj]["annot"].index = data[subj]["exprs"].columns

file_rawdata = './AHBA_processing_results/rawdata.pkl'
pickle.dump([probe_annot, probe_reannot, data], open(file_rawdata, 'wb'))

# 1.Filtering samples and probes

In [159]:
# quick load
file_rawdata = './AHBA_processing_results/rawdata.pkl'
probe_annot, probe_reannot, data = pickle.load(open(file_rawdata, 'rb'))
nSUBJ = len(data)

In [160]:
# re-annotation
probe_reannot_filter1 = pd.merge(probe_reannot[['probe_name', 'gene_symbol', 'entrez_id']], 
                                 probe_annot[["probe_name", "probe_id"]], 
                                 on="probe_name", how="left").set_index("probe_id").dropna(subset=["entrez_id"])
probe_reannot_filter1.loc[:, "entrez_id"] = probe_reannot_filter1["entrez_id"].astype(int)

In [161]:
# 1.1 remove sample in Brainstem(BS) and Cerebellum(CB), select reannotated probes
index_probe_keep = probe_reannot_filter1.index
for subj in range(1, nSUBJ+1):
    index_sample_keep   = ~data[subj]['annot'].slab_type.isin(["BS", "CB"])
    data[subj]['exprs'] = data[subj]['exprs'].loc[index_probe_keep, index_sample_keep]
    data[subj]['noise'] = data[subj]['noise'].loc[index_probe_keep, index_sample_keep]
    data[subj]['annot'] = data[subj]['annot'].loc[index_sample_keep, :]

In [162]:
# 1.2 probe: intensity-based filtering (IBF)
threshold = 0.5
signal_level, n_sample = np.zeros(probe_reannot_filter1.shape[0]), 0
for subj in range(1, nSUBJ+1):
    signal_level += data[subj]['noise'].sum(axis=1)
    n_sample += data[subj]['noise'].shape[1]
index_probe_keep_IBF = (signal_level / n_sample ) > threshold

probe_reannot_filter2 = probe_reannot_filter1.loc[index_probe_keep_IBF, :]
for subj in range(1, nSUBJ+1):
    data[subj]['exprs'] = data[subj]['exprs'].loc[index_probe_keep_IBF, :]
    data[subj]['noise'] = data[subj]['noise'].loc[index_probe_keep_IBF, :]

In [163]:
# 1.3 probe: select representative probe (DS method)
region_exprs = [ data[subj]['exprs'].groupby(data[subj]['annot'].structure_id, axis=1).mean().T.rank() 
                 for subj in range(1, nSUBJ+1) ]  # sample * probe

## calc DS score for each probes
ds_score = np.zeros(probe_reannot_filter2.shape[0])
for i in range(len(region_exprs)-1):
    exprs1_zscore = (region_exprs[i] - region_exprs[i].mean(axis=0)) / region_exprs[i].std(axis=0)
    for j in range(i+1, len(region_exprs)):
        exprs2_zscore = (region_exprs[j] - region_exprs[j].mean(axis=0)) / region_exprs[j].std(axis=0)
        samples = np.intersect1d(exprs1_zscore.index, exprs2_zscore.index)
        ds_score += (exprs1_zscore.loc[samples, :] * exprs2_zscore.loc[samples, :]).sum(axis=0) / (len(samples) - 1)
ds_score /= sum(range(len(region_exprs)))

## select probe
max_ds_idx = pd.DataFrame([ds_score, probe_reannot_filter2.entrez_id]).T.reset_index().set_index(keys=["entrez_id", "probe_id"]).groupby("entrez_id").idxmax()["Unnamed 0"]
index_probe_keep_DS =  pd.Index(max_ds_idx.apply(lambda x:x[1]).values)
probe_reannot_filter3 = probe_reannot_filter2.loc[index_probe_keep_DS, :]
for subj in range(1, nSUBJ+1):
    data[subj]['exprs'] = data[subj]['exprs'].loc[index_probe_keep_DS, :]
    data[subj]['noise'] = data[subj]['noise'].loc[index_probe_keep_DS, :]

In [165]:
file_processed_data = "./AHBA_processing_results/01.processed_AHBA.pkl"
pickle.dump([probe_reannot_filter3, data], open(file_processed_data, 'wb'))

# 2.Assign samples to parcellation